# 210C (Jonannes) Homework 2 by Mikey

_I use R for the code in this assignment, which is not built into VSCode by default. In order to run this code, you may need to download R, add the R extension to VSCode, and then run the following in a terminal:_

```
R -e 'install.packages("IRkernel", repos="https://cloud.r-project.org/")'
```

## VARs
### Download data for the Federal Funds Rate, the civilian unemployment rate, and the GDP deflator inflation rate from FRED.

In [ ]:
# Clear all.
rm(list = ls())

install.packages("quantmod")
library(quantmod)

# Import Federal Funds Rate, Unemployment Rate, and GDP Deflator Inflation rate data from FRED.
R <- getSymbols("FEDFUNDS", src = "FRED", auto.assign = FALSE)
u <- getSymbols("UNRATE", src = "FRED", auto.assign = FALSE)
pi <- getSymbols("GDPDEF", src = "FRED", auto.assign = FALSE)

#Put the data into data frames.
R <- data.frame(date = index(R), R = coredata(R)[, "FEDFUNDS"])
u <- data.frame(date = index(u), u = coredata(u)[, "UNRATE"])
pi <- data.frame(date = index(pi), pi = coredata(pi)[, "GDPDEF"])

### (a) Plot the data. Make sure all graphs are appropriately labelled.

In [ ]:
install.packages("ggplot2")
library(ggplot2)

ggplot(R, aes(x = date, y = R)) + geom_line() + labs(title = "Federal Funds Rate", x = "Date", y = "Percentage Points")
ggplot(u, aes(x = date, y = u)) + geom_line() + labs(title = "Unemployment Rate", x = "Date", y = "Percentage Points")
ggplot(pi, aes(x = date, y = pi)) + geom_line() + labs(title = "GDP Deflator Inflation Rate", x = "Date", y = "Percentage Points")


### (b) Aggregate all series to a quarterly frequency by averaging over months.

In [ ]:
R <- aggregate(R$R, list(yearqtr = as.yearqtr(R$date)), mean)
u <- aggregate(u$u, list(yearqtr = as.yearqtr(u$date)), mean)
pi <- aggregate(pi$pi, list(yearqtr = as.yearqtr(pi$date)), mean)

### Estimate a VAR with 4 lags from 1960Q1:2007Q4. The ordering of your variables should be $\pi_t,u_t,R_t$.

In [ ]:
install.packages("vars")
library(vars)

# Merge the data into a single data frame
data <- merge(pi, merge(u, R, by = "yearqtr"), by = "yearqtr")
colnames(data) <- c("yearqtr", "pi", "u", "R")

# Calculate inflation
library(dplyr)
data$pi <- 100 * (data$pi - lag(data$pi, 4)) / lag(data$pi, 4)

# Filter the data to keep observations from 1960Q1 to 2007Q4
start_date <- as.yearqtr("1960 Q1")
end_date <- as.yearqtr("2007 Q4")
data_filtered <- data[data$yearqtr >= start_date & data$yearqtr <= end_date, ]

# Convert the filtered data to a time series object with quarterly frequency
time_series <- ts(data_filtered[, c("pi", "u", "R")], start = c(1960, 1), end = c(2007, 4), frequency = 4)

# Estimate the VAR model with 4 lags
var_model <- VAR(time_series, p = 4, type = "const")

### (c) Briefly, explain why it would make sense to end the sample in 2007Q4?
That is the onset of the Great Recession, which is probably the most significant economic event since the Great Depression. If we want to test the external validity of our VAR model, we can train it on data from "steady state" (pre-2007) and see how well it performs out of sample. Also, there was some pretty big government expenditure in response to the Great Recession, so if we train the VAR using data from that time, it may be capturing the response of the variables to this government activity (an omitted variable) rather than to natural fluctuations to the rest of the three (included) variables.

### (d) Plot the IRFs from the SVAR with the same ordering. [Optional: add 95\% error bands]

In [ ]:
# Create matrices for ordering
A <- matrix(c(NA, 0, 0, 
               NA, NA, 0,
               NA, NA, NA), nrow = 3, ncol = 3, byrow = TRUE)

B <- diag(3)

# Estimate the SVAR model using the VAR model and identification restrictions
svar <- SVAR(var_model, Amat = A, Bmat = B, estmethod = "direct", lrtest = FALSE)

#Calculate the Impulse Response Functions (IRFs) with bootstrapped standard errors
irf <- irf(var_model, impulse = c("pi", "u", "R"), response = c("pi", "u", "R"), boot = TRUE, runs = 1000)

# Set the background color to white for legibility
par(bg = "white")

# Plot each combination of impulse and response
plot(irf, lwd = 3, bg = "white")

### (e) Briefly, interpret your results.
Shocks to inflation preceeds...
* Sustained, elevated inflation;
* Slowly increasing unemployment;
* Sustained, elevated interest rates.
    
The story here sounds like that of government response: if inflation is high, the Fed responds by increasing interest rates, surpressing economic activity.

Shocks to unemployment preceeds...
* A slow dip and then slow rebound in inflation;
* A slow dissipation of the unemployment shock;
* A sharp decrease and slow regression of interest rates.

If unemployment increases, then spending should decrease, slowing inflation. The unemployment shock is somewhat sticky, so the government responds by lowering interest rates.

Shocks to interest rates preceeds...
* Remarkably little movement in inflation;
* A small, slow increase in unemployment;
* A steady regression of interest rates down to the mean.

This speaks to the tricky reverse causaility story. Increased interest rates are supposed to cool down an economny and prevent inflation, but if they are often used as a tool by the Fed to counteract anticipated inflation and an "overheating" economy, the causal effect and the reverse causal effect may cancel each other out entirely.

### (f) Plot the time series of your identified monetary shocks.

In [ ]:
# Extract the residuals (shocks) from the VAR model
residuals <- residuals(var_model)

# Adjust the dates to match the residuals (drop the initial lags)
adjusted_dates <- data_filtered$yearqtr[-(1:4)]

# Convert residuals to a data frame for plotting
residuals_df <- data.frame(date = adjusted_dates, residuals)

# Rename the columns to match the variable names
colnames(residuals_df) <- c("date", "pi_shock", "u_shock", "R_shock")

# Set up a 3x1 plotting space
par(mfrow = c(3, 1), mar = c(4, 4, 2, 1), bg = "white")  

# Plot each shock time series
plot(residuals_df$date, residuals_df$pi_shock, type = "l", col = "red", 
     xlab = "", ylab = "Percentage Points", main = "Deviation in Inflation (pi) from VAR Prediction")
plot(residuals_df$date, residuals_df$R_shock, type = "l", col = "blue", 
     xlab = "", ylab = "Percentage Points", main = "Deviation in Federal Funds Rate (R) from VAR Prediction")
plot(residuals_df$date, residuals_df$u_shock, type = "l", col = "green", 
     xlab = "", ylab = "Percentage Points", main = "Deviation in Unemployment (u) from VAR Prediction")

### (g) What are the identified monetary shocks in 2001Q3 and 2001Q4? How should one interpret these shocks?
This is the aftermath of the September 11 attacks. To be honest, I don't see much happening in my plots, which makes me fearful that I did something wrong — inflation seems to be in line with the predictions of the model, and although unemployment is higher than predicted, the residual is not more than 0.5 percentage points. There is a somewhat notable dip in interest rates, which suggests to me that the government responded to the attacks by lowering interest rates.

Although I was old enough to remember this, I was not old enough to care about what the government did in response. A brief skim of the [September 11 attacks Wikipedia page](https://en.wikipedia.org/wiki/Economic_effects_of_the_September_11_attacks) reveals that unemployment increased sharply in New York City, which would have a small but not imperceptable effect on the overall US economy and explains our result. Meanwhile, stock markets took a massive hit around the world, which explains the government response that we see.


## Romer shocks
### (a) Download the Romer–Romer shocks from my website and merge it with your VAR dataset. Set the values of the Romer shocks to zero before 1969Q1.

In [ ]:
setwd("/Users/mikey/Documents/Academic/UCSD/Macro 210C/Macro210C/HW2/")
install.packages("haven")
library(haven)
romer <- read_dta("RR_monetary_shock_quarterly.dta")

# Convert the date variable to year-quarter format.
start_date <- as.yearqtr("1960 Q1")  # Adjust the start date if necessary
romer$yearqtr <- as.yearqtr(start_date + romer$date/4)

# Merge the datasets.
data_merged <- merge(data_filtered, romer, by = "yearqtr", all.x = TRUE)

# Rename resid_romer to RR
colnames(data_merged)[colnames(data_merged) == "resid_romer"] <- "RR"

# Set the values of the Romer shocks to zero before 1969Q1.
data_merged$RR[data_merged$yearqtr < as.yearqtr("1969 Q1")] <- 0

### (b) Following Romer–Romer, construct the IRF from the estimation equation
$$
    \begin{equation*}
            y_t = \alpha + \sum_{s=1}^{8}\beta_s y_{t-s} + \sum_{s=0}^{12}\gamma_{s}RR_{t-s}
    \end{equation*}
$$
### where $y_t\in[\pi_t,u_t,R_t]$ are the outcome variables and $RR_t$ are the Romer shocks estimated from 1960Q1:2007Q4. [Optional: add 95\% error bands]

In [ ]:
# Generate 12 lagged variables for RR using a loop
for (i in 1:12) {
  data_merged[[paste0("RR_", i)]] <- lag(data_merged$RR, i)
}

# Replace NA values in data_merged with 0
data_merged[is.na(data_merged)] <- 0

# Define RR_lags as a vector of RR and all its lags.
RR_lags <- c("RR", paste0("RR_", 1:12))

# Convert RR_lags to a matrix and replace NAs with zeros
RR_lags_mat <- as.matrix(data_merged[, RR_lags])
RR_lags_mat[is.na(RR_lags_mat)] <- 0

# Create time series with variables in correct order.
time_series_romer <- ts(data_merged[, c("pi", "u", "R")], start = c(1960, 1), end = c(2007, 4), frequency = 4)
time_series_romer[is.na(time_series_romer)] <- 0

# VAR it up.
var_romer <- VAR(time_series_romer, p = 8, type = "const", exogen = RR_lags_mat)

#IRFs (with bootstrapped standard errors)
irf_romer <- irf(var_romer, impulse = c("pi", "u", "R"), response = c("pi", "u", "R"), boot = TRUE, runs = 1000, n.ahead = 10)
par(bg = "white")
plot(irf_romer, lwd = 3)

### (c) Now estimate an SVAR ordered $RR_t,\pi_t,u_t,R_t$ with four lags from 1960Q1:2007Q4 and plot the IRFs. [Optional: add 95\% error bands]

In [ ]:
# Create time series with variables in correct order.
time_series_romer <- ts(data_merged[, c("RR", "pi", "u", "R" )], start = c(1960, 1), end = c(2007, 4), frequency = 4)

# Estimate the VAR model on which this SVAR will be based.
var_romer <- VAR(time_series_romer, p = 4, type = "const")

# Create matrices for ordering.
AA <- matrix(c(NA, 0, 0, 0, 
               NA, NA, 0, 0, 
               NA, NA, NA, 0, 
               NA, NA, NA, NA), nrow = 4, ncol = 4, byrow = TRUE)

BB <- diag(4)

# Estimate the SVAR model using the VAR model and identification restrictions.
svar_romer <- SVAR(var_romer, Amat = AA, Bmat = BB, estmethod = "direct", lrtest = FALSE)

#IRFs (with bootstrapped standard errors)
irf_svar_romer <- irf(svar_romer, impulse = c("pi", "u", "R", "RR"), response = c("pi", "u", "R", "RR"), boot = TRUE, runs = 1000, n.ahead = 10)
par(bg = "white")
plot(irf_svar_romer, lwd = 4)

### (d) Briefly, explain why it is sensible to order the Romer shock first in the VAR.
Romer and Romer (2023) documents changes in monetary policy that are "not taken in response to current or prospective developments in real activity." As we are on a quest for exogenous shocks to interest rates, this is probably as close as we're ever going to come as shocks that are uncorrelated with the observed variables in this dataset. I.e., it is plausible that we avoid the reverse causality problem, where $\pi_t$, $u_t$, or $R_t$ causes $RR_t$. Meanwhile, we take the stance that $RR_t$ is allowed to cause $\pi_t$, $u_t$, or $R_t$ to change. That seems reasonable.

### (e) Compare the IRFs for the Romer shocks from the two methods. How are they different, and why?
Note: I can't get this package to give me the IRF to exogenous variables, which I know is the entire point. I tried another package, and it went even worse. Oh well. Based on my results, the outputs are remarkably similar. There are two main differences, as far as I can tell:

1) The SVAR responses are smoother. 
2) The SVAR responses are much larger in magnitude.

I think the smoothness is an artifact of the magnitude thing, and is not a deep result. The magnitude, however, is interesting _per se_. This tells me that the coefficients on lagged $RR_t$ are soaking up quite a bit of the variance in part (b).

### (f) Compare the VAR IRFs for the Romer shocks with the VAR IRFs for the SVAR shocks in Question (1d). How are they different, and why?
Again, the magnitudes appear a bit bigger. But the biggest thing I see is that in response to a shock to $R_t$, there is a huge positive response of $\pi_t$ in the Romer shocks models. This is, I think, the opposite of what I would expect if the Romer model were better at absorbing the endogeneity inherent in the forward-looking behavior of the Fed. Right? Raising interest rates should cause inflation to decrease, not increase.

### (g) Compare the Romer-Romer the identified monetary shocks in 2001Q3 and 2001Q4 with the SVAR identified monetary shocks. How are they similar / different?


In [ ]:
# Extract the residuals (shocks) from the VAR model
residuals_romer <- residuals(var_romer)

# Convert residuals to a data frame for plotting
residuals_romer_df <- data.frame(date = adjusted_dates, residuals_romer)

# Rename the columns to match the variable names
colnames(residuals_romer_df) <- c("date", "pi_shock", "u_shock", "R_shock")

# Set up a 3x1 plotting space
par(mfrow = c(3, 1), mar = c(4, 4, 2, 1), bg = "white")  

# Plot each shock time series
plot(residuals_romer_df$date, residuals_romer_df$pi_shock, type = "l", col = "red", 
     xlab = "", ylab = "Percentage Points", main = "Deviation in Inflation (pi) from VAR Prediction")
plot(residuals_romer_df$date, residuals_romer_df$R_shock, type = "l", col = "blue", 
     xlab = "", ylab = "Percentage Points", main = "Deviation in Federal Funds Rate (R) from VAR Prediction")
plot(residuals_romer_df$date, residuals_romer_df$u_shock, type = "l", col = "green", 
     xlab = "", ylab = "Percentage Points", main = "Deviation in Unemployment (u) from VAR Prediction")

Now the interest rate is actually goes up instead of down. This suggests to me that the response to September 11 by the Fed is actually being captured as a Romer shock — which suggests that they are responding to something other than observables in this case.